# Lineare Regression & Regularisierung

In diesem Notebook baue ich ein erstes **lineares Modell** zur Vorhersage des Verkaufspreises von Gebrauchtwagen.  
Ziel ist es, eine **interpretierbare Baseline** zu erstellen und zu untersuchen, wie gut sich der Preis mit einer (regularisierten) linearen Regression erklären lässt.

Im Gegensatz zu komplexeren Modellen (z. B. Gradient Boosting) liegt der Fokus hier weniger auf maximaler Performance, sondern auf dem Gewinnen von Einblicken in die Preisbildung der Gebrauchtwagen.

## Gliederung

In diesem Notebook gehe ich wie folgt vor:

### 1. Daten laden & Überblick
- Importieren der benötigten Bibliotheken
- Laden des vorbereiteten Datensatzes
- Kurzer Überblick über den Datensatz

### 2. Baseline-Modell (OLS)
- Definition einer Funktion für lineare Regression
- Auswahl eines Feature-Sets durch iteratives Entfernen

### 3. Erstellung neuer Features
- RTO-Features
- Car Rating als numerisches Feature
- Interaktionsfeatures

### 4. Binning kategorialer Features
- Motivation und Einordnung des Binning-Ansatzes
- Definition einer Funktion zur Bestimmung geeigneter Bins
- Anwendung des Binning auf ausgewählte Features

### 5. Logarithmierung des Targets
- Motivation für die Log-Transformation
- Definition einer linearen Regression mit logarithmiertem Target
- Iteratives Hinzufügen von Features

### 6. Ridge-Regression
- Implementierung einer Ridge-Regression
- Hyperparameter-Tuning
- Feature-Selection unter Regularisierung

### 7. Übersicht & Interpretation
- Vergleich der Modellperformance
- Interpretation kategorialer Koeffizienten
- Interpretation numerischer Koeffizienten

### 8. Fazit
- Zusammenfassung der Modellschritte
- Einordnung der Ergebnisse
- Ableitung einer linearen Baseline für weiterführende Modelle

Am Ende dieses Notebooks steht eine gut dokumentierte, interpretierbare lineare Baseline,
die als Referenz für komplexere Modelle in den folgenden Schritten dient.


## Daten Laden und Überblick
#### Importieren der benötigten Bibliotheken

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.compose import ColumnTransformer, make_column_selector as Selector
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV, Ridge, Lasso
from prettytable import PrettyTable
from sklearn.pipeline import Pipeline
import time
import random

random.seed(42)
np.random.seed(42)

#### Laden des vorbereiteten Datensatzes + Kurzer Blick

In [7]:
data_dir = Path('data/processed')
data_file = data_dir / 'cleaned_data_02.csv'
df = pd.read_csv(data_file)
print(df.shape)
df.head()

(7391, 17)


,fuel_type,kms_run,sale_price,city,body_type,transmission,variant,registered_city,registered_state,rto,make,model,total_owners,car_rating,fitness_certificate,warranty_avail,age
0,petrol,8063,386399,noida,hatchback,manual,lxi opt,delhi,delhi,dl6c,maruti,swift,2,great,True,False,6
1,petrol,23104,265499,noida,hatchback,manual,lxi,noida,uttar pradesh,up16,maruti,alto 800,1,great,True,False,5
2,petrol,23402,477699,noida,hatchback,manual,sports 1.2 vtvt,agra,uttar pradesh,up80,hyundai,grand i10,1,great,True,False,4
3,diesel,39124,307999,noida,hatchback,manual,vdi,delhi,delhi,dl1c,maruti,swift,1,great,True,False,8
4,petrol,22116,361499,noida,hatchback,manual,magna 1.2 vtvt,new delhi,delhi,dl12,hyundai,grand i10,1,great,True,False,6


## Baseline-Modell (OLS)

#### Definition einer Funktion für lineare Regression

Um neue Features effizient bewerten zu können, definiere ich zunächst eine kompakte
Baseline-Regression, die automatisch One-Hot-Encoding und Skalierung übernimmt.
Die Funktion liefert einen schnellen Cross-Validation-Score, mit dem ich
verschiedene Feature-Sets vergleichen kann.


In [8]:
def get_regression(data, return_mae = False):
    X = data.copy()
    y = X.pop('sale_price')
    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), Selector(dtype_include = 'number')),
        ('cat', OneHotEncoder(drop = 'first', handle_unknown = 'ignore'), Selector(dtype_include = 'object'))
    ])

    pipe = make_pipeline(preprocessor, LinearRegression())

    score_r2 = cross_val_score(pipe, X, y, cv = 5)

    if return_mae:
        score_mae = cross_val_score(pipe, X, y, cv = 5, scoring = 'neg_mean_absolute_error')
        score_mae = - score_mae
        return score_r2.mean(), score_mae.mean()
    else:
        return score_r2.mean()


In [9]:
get_regression(df)

np.float64(0.8714220281142664)

#### Auswahl eines Featureset durch iteratives Entfernen
Eine simple Methode für die Auswahl eines kompakten Featuresets, ist es, je ein Feature zu entfernen und darauf die Regression zu trainieren. Dasjenige Feature, dessen Elimination zum höchsten Performancezuwachs führt, wird entfernt. Dies wird wiederholt bis keine Zuwächse mehr erreicht werden können. Dies führt zu einem lokalen Optimun, was hier genügen soll.

In [10]:
for feature in df.drop(columns = 'sale_price').columns:
    print('Ohne', feature, ':', get_regression(df.drop(columns = feature)))

Ohne fuel_type : 0.8714499177471632
Ohne kms_run : 0.8667705782199893
Ohne city : 0.87198900156193
Ohne body_type : 0.8723363040434536
Ohne transmission : 0.8693202797927299
Ohne variant : 0.8230952019775067
Ohne registered_city : 0.8738443874505192
Ohne registered_state : 0.8702762004219176
Ohne rto : 0.8741444240249759
Ohne make : 0.8544429162210314
Ohne model : 0.8366726851232376
Ohne total_owners : 0.8699189127833824
Ohne car_rating : 0.8696160392092969
Ohne fitness_certificate : 0.8714220281142664
Ohne warranty_avail : 0.8714220281142664
Ohne age : 0.8226335630065963


In [11]:
base = ['kms_run', 'sale_price', 'transmission', 'variant', 'registered_state', 'make', 'model', 'total_owners', 'car_rating', 'age']
result_1 = get_regression(df[base], return_mae = True)
result_1

(np.float64(0.8775428447169207), np.float64(49339.719960399234))

Weiter Iterationen ergaben ein lokales Optimum der Regression mit den Features: **'kms_run', 'sale_price', 'transmission', 'variant', 'registered_state', 'make', 'model', 'total_owners', 'car_rating', 'age'**.

## Erstellung neuer Features


In [12]:
#Kopie des Datensatzes. Änderungen werden auf der Kopie durchgeführt.
df_fe = df.copy()

#### RTO-Features
Die RTO-Kennung besteht aus zwei Buchstaben (Bundesstaat) und einer Zahlenfolge. Um die hohe Kardinalität zu reduzieren, trenne ich die Variable in zwei Teile:  

- **rto_state**: zweibuchstabiges Zeichen für Bundesstaat ähnlich zu registered_state
- **rto_number**: numerische weitere Unterteilung der Bundestaaten, vermutlich weniger relevant

In [13]:
df_fe['rto_state'] = df_fe['rto'].astype(str).str[:2]
df_fe['rto_number'] = df_fe['rto'].astype(str).str[2:]

In [14]:
features = base + ['rto_state', 'rto_number']
print(get_regression(df[base]))
print(get_regression(df_fe[features]))
print(get_regression(df_fe[features].drop(columns = 'rto_number')))
print(get_regression(df_fe[features].drop(columns = 'rto_state')))
print(get_regression(df_fe[features].drop(columns = 'registered_state')))
print(get_regression(df_fe[features].drop(columns = ['rto_number', 'registered_state'])))

0.8775428447169207
0.8761836073001856
0.8776947371383731
0.8759707388477673
0.8759557366726677
0.8776710754218247


rto_number scheint überflüssig zu sein. Mit registered_state und rto_state performt das Modell minimal besser.

#### Car_rating als numerisches Feature
Da car_rating eine logische Reihenfolge besitzt, stelle ich car_rating als Integer da.

In [15]:
df_fe['car_rating_number'] = df_fe.car_rating.map({'overpriced' : 1, 'fair' : 2, 'good' : 3, 'great' : 4})

In [16]:
features = base + ['car_rating_number']
print(get_regression(df[base]))
print(get_regression(df_fe[features]))
print(get_regression(df_fe[features].drop(columns = 'car_rating')))

0.8775428447169207
0.8775435681663609
0.8773592439514809


Auch Car_rating_number verbessert die lineare Regression noch nicht nennenswert.

#### Interaktionsfeatures
Interaktionsfeatures ermöglichen es linearen Modellen komplexe Zusammenhänge besser zu erfassen. Erstellen wir einge Verhältnisse, wobei eine Vielzahl an anderen Interaktionen denkbar sind.

In [17]:
#KM pro Besitzer
df_fe['kms_per_owner'] = df_fe['kms_run'] / df_fe['total_owners']

#KM pro Jahr/ Wie viel wird Auto gefahren?
df_fe['kms_per_age'] = np.where(df_fe['age'] == 0, df_fe['kms_run'], df_fe['kms_run'] / df_fe['age'])

#Durchschnittliche KM pro Modell
df_fe['avg_kms_by_model'] = df_fe.groupby('model')['kms_run'].transform('mean')

#KM im Vergleich zu anderen Autos des gleichen Modells
df_fe['kms_vs_model_avg'] = df_fe['kms_run'] / df_fe['avg_kms_by_model']

# Durchschnittliches Alter pro Modell
df_fe['avg_age_by_model'] = df_fe.groupby('model')['age'].transform('mean')

#Alter im Vergleich zu anderen Autos des gleichen Modells
df_fe['age_vs_model_avg'] = df_fe['age'] / df_fe['avg_age_by_model']

#Druchschnittliches Rating pro Modell
df_fe['avg_rating_by_model'] = df_fe.groupby('model')['car_rating_number'].transform('mean')

#Rating im Vergleich zu anderen Autos des gleichen Modells
df_fe['rating_vs_model_avg'] = df_fe['car_rating_number'] / df_fe['avg_rating_by_model']

In [18]:
ia_features =  ['kms_per_owner', 'kms_per_age', 'avg_kms_by_model', 'kms_vs_model_avg',
       'avg_age_by_model', 'age_vs_model_avg', 'avg_rating_by_model', 'rating_vs_model_avg']

print(result_1)
for i in ia_features:
    features = base + [i]
    print(i, ':', get_regression(df_fe[features]))

(np.float64(0.8775428447169207), np.float64(49339.719960399234))
kms_per_owner : 0.8775210442242202
kms_per_age : 0.8775125041728818
avg_kms_by_model : 0.8771324074646165
kms_vs_model_avg : 0.8780795942341172
avg_age_by_model : 0.8782374298784585
age_vs_model_avg : 0.8770883887929461
avg_rating_by_model : 0.877830884358245
rating_vs_model_avg : 0.8776140526687216


Iteratives Hinzufügen ergabe ein lokales Optimum mit folgenden zusätzlichen Features:  

**'kms_vs_model_avg'**, **'avg_age_by_model'**, **'age_vs_model_avg'**  

In [19]:
features = base + ['kms_vs_model_avg', 'avg_age_by_model','age_vs_model_avg']
result_2 = get_regression(df_fe[features], return_mae = True)
result_2

(np.float64(0.8787769742127063), np.float64(49051.46870153751))

Der Nutzen unserer Zusätzlichen Features bleibt insgesamt relativ gering für unsere einfache Lineare Regression. Dies kann sich jedoch bei den folgenden modifizierten Regressionen ändern.

## Binning kategorischer Features

#### Defnintion einer Funktion für optimale Binsize pro Feature

Hier versuche ich die Modelperformance zu verbessern, indem ich systemsatisch optimale Grenzen finde, bei denen ich die Kategorien zusammenfasse zu einer neuen Kategorie "OTHER". Dafür wird ein Feature ausgewählt und für unterschiedliche Tresholds je eine Regression trainiert, wobei aus dem höchsten Score die optimale Grenze geschlossen wird.

In [20]:
def optimal_bin(df, feature, max_thr = 30):
    table = PrettyTable()
    table.field_names = ['min_bin_size', 'r2']

    vc = df[feature].value_counts()
    thresholds = sorted(vc[vc <= max_thr].unique())

    for t in thresholds:
        df_base = df.copy()
        rare = vc[vc < t].index
        df_base[feature] = np.where(df_base[feature].isin(rare), 'OTHER', df_base[feature])
        r2 = get_regression(df_base[features])
        table.add_row([t, r2])

    return table

In [21]:
optimal_bin(df_fe, 'make')

min_bin_size,r2
1,0.8787769742127063
2,0.8813791478880937
3,0.8813617866813317
9,0.8813424146913171
13,0.8809761277141821
15,0.8810583958306684
27,0.8796986915046394


Für das Feature 'make' erreichen wir den besten R2-Wert, wenn wir lediglich Kategorien mit je einer Ausprägung zusammenfassen.  
Anwendung der Funktion auf kategorische Features ergab folgende optimale Tresholds:
- model : 2
- registered_state : 22
- make : 2
- variant: 1

Bei variant ist also die minimale Anzahl die eine Kategorie haben muss um nicht zu OTHER zusammengefasst zu werden 1. Dies ist überraschend, da das Model bei einer Kategorie, die im Testsatz einmal vorkommt, keine Informationen aus dem Trainingssatz ziehen kann. Der Performanceverlust scheint also durch Leakage zu entstehen. Mögliche Lösungen wäre es Gruppen nach dem Split zusammenzufassen.

In [22]:
# model binnen
counts = df_fe.model.value_counts()
freq = counts[counts >= 2]
df_fe['model'] = df_fe['model'].apply(lambda x: x if x in freq else 'Other')

# registered_state binnen
counts = df_fe.registered_state.value_counts()
freq = counts[counts >= 22]
df_fe['registered_state'] = df_fe.registered_state.apply(lambda x: x if x in freq else 'Other')

# make binnen
counts = df_fe.make.value_counts()
freq = counts[counts >= 2]
df_fe['make'] = df_fe['make'].apply(lambda x: x if x in freq else 'Other')

In [23]:
result_3 = get_regression(df_fe[features], return_mae = True)
result_3

(np.float64(0.8817925543912537), np.float64(48853.277375963844))

## Logarithmieren des Targets
Da das Target eine Schiefe Verteilung hat, könnte Logarithmieren des Targets eine Verbesserung der Modellperformance erwirken.

In [24]:
def log_reg(data, cv = 5, return_mae = False):
    X = data.drop(columns = 'sale_price')
    y = data.sale_price

    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), Selector(dtype_include = 'number')),
        ('kat', OneHotEncoder(drop = 'first', handle_unknown = 'ignore'), Selector(dtype_include ='object'))
    ])
    model = LinearRegression()
    reg = TransformedTargetRegressor(
        regressor = model,
        func = np.log1p,
        inverse_func = np.expm1
    )
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', reg)
    ])
    score_r2 = cross_val_score(pipe, X, y, cv = cv)
    if return_mae:
        score_mae = cross_val_score(pipe, X, y, cv = cv, scoring = 'neg_mean_absolute_error')
        score_mae = -score_mae
        return score_r2.mean(), score_mae.mean()
    return score_r2.mean()

In [27]:
log_reg(df_fe[base])

np.float64(0.8909684070843774)

#### Iteratives Hinzufügen von Features
Der Score ist bereits besser, doch die Logarithmierung des Targets kann es ermöglichen noch mehr Features gewinnbringend einzusetzen. Ich füge iterativ Features zu unserem bisherigen Set hinzu, um ein gutes Featureset zu finden.

In [28]:
out = [c for c in df_fe.columns if c not in base]
for o in out:
    features = base + [o]
    print(o, ':', log_reg(df_fe[features]))

fuel_type : 0.8920306894677283
city : 0.8902615504147576
body_type : 0.8885017269948354
registered_city : 0.8879359673810349
rto : 0.8811910526022508
fitness_certificate : 0.8909684070843774
warranty_avail : 0.8909684070843774
rto_state : 0.891951610184336
rto_number : 0.8886915176964214
car_rating_number : 0.8909704266591391
kms_per_owner : 0.8909409598412917
kms_per_age : 0.8907515415549586
avg_kms_by_model : 0.8905700600668626
kms_vs_model_avg : 0.8904699379861631
avg_age_by_model : 0.8882492174215951
age_vs_model_avg : 0.8926453679728297
avg_rating_by_model : 0.8906567585702779
rating_vs_model_avg : 0.8891905589268287


Ein Lokoales Optimum ergibt sich somit mit den zusätzlichen Features:
**'age_vs_model_avg', 'body_type', 'avg_kms_by_model', 'kms_vs_model_avg', 'fuel_type', 'avg_age_by_model'**.

In [29]:
opt_features = base + ['fuel_type', 'age_vs_model_avg', 'rto_state', 'kms_per_age']
result_4 = log_reg(df_fe[opt_features], return_mae = True)
result_4

(np.float64(0.8952103164008044), np.float64(47026.38568074534))

## Regularierte Modelle
Durch die Einführung eines Strafterms verringert Regularisierung Overfitting auf den Trainingsdaten. Durch eine verbesserte Generalisierung kann oft eine bessere Erklärungkraft erzielt werden. Dies gilt insbesondere für Daten mit hochkardinalen Features, für uns also perfekt.

#### Funktion für Ridge-Regression
Im nächsten Schritt erweitere ich das lineare Modell um eine L2-Regularisierung (Ridge). Dabei wird ein Strafterm eingeführt, was die Koeffizienten reduziert und zu geringerem Overfitting führt, insbesondere bei stark korrelierten Features.

Die folgende Funktion trainiert das Ridge-Modell und kann auf Wunsch zusätzlich die Referenzkategorien unserer kategorischen Features, sowie die Modellkoeffizienten zurückgeben.

In [30]:
def get_ridge(data, alpha = 1, return_reference = False, return_coef = False, return_mae = False):
    X = data.copy()
    y = X.pop('sale_price')
    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), Selector(dtype_include = 'number')),
        ('cat', OneHotEncoder(drop = 'first', handle_unknown = 'ignore', sparse_output=False),
         Selector(dtype_include = 'object'))
    ])

    ridge = Ridge(alpha = alpha)

    reg = TransformedTargetRegressor(
        regressor=ridge,
        func=np.log1p,
        inverse_func=np.expm1,
    )

    pipe = make_pipeline(preprocessor, reg)

    score_r2 = cross_val_score(pipe, X, y, cv=5)

    if return_reference | return_coef:
        pipe.fit(X,y)
        pre = pipe.named_steps['columntransformer']

    if return_reference:
        ohe = pre.named_transformers_['cat']
        cat_cols = X.select_dtypes('object').columns

        print('Referenzkategorien:')
        reference = []
        for col, cats in zip(cat_cols, ohe.categories_):
            ref = cats[0]
            dummies = list(cats[1:])
            reference.append((col, ref))
        reference_df = pd.DataFrame(reference, columns = ['Feature', 'Referenzkategorie'])
        reference_df.index += 1


    if return_coef:
        feature_names = pre.get_feature_names_out()

        tt = pipe.named_steps['transformedtargetregressor']
        ridge = tt.regressor_
        coef = pd.Series(ridge.coef_, index=feature_names)

        important = coef.reindex(coef.abs().sort_values(ascending=False).index)
        return important

    elif return_reference:
        return reference_df

    elif return_mae:
        score_mae = cross_val_score(pipe, X, y, cv = 5, scoring = 'neg_mean_absolute_error')
        score_mae = -score_mae
        return score_r2.mean(), score_mae.mean()

    else:
        return scores.mean()

#### Funktion für Hyperparameter
Die folgende Funktion verwednet GridSearchCV, um den optimalen Wert für den Hyperparameter **alpha** für unsere Regression zu ermitteln, dabei ist es möglich einen oberen und unteren startwert, sowie die Anzahl der Werte, die probiert werden anzugeben.

In [31]:
def get_alpha_ridge(data, low = -4, high = 1, n = 50):
    X = data.copy()
    y = X.pop('sale_price')

    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), Selector(dtype_include = 'number')),
        ('car', OneHotEncoder(drop = 'first', handle_unknown = 'ignore', sparse_output = False),
         Selector(dtype_include = 'object'))
    ])

    regressor = Ridge()
    reg = TransformedTargetRegressor(
        regressor = regressor,
        func = np.log1p,
        inverse_func = np.expm1
    )

    pipe = make_pipeline(preprocessor, reg)
    param_grid = {
    'transformedtargetregressor__regressor__alpha': np.logspace(low, high, n)
    }
    search = GridSearchCV(pipe, param_grid, cv = 5)
    search.fit(X, y)
    return search.best_params_

#### Funktion für Feature Selection
Zur Auswahl geeigneter Eingangsdaten verwende ich ein einfaches
Forward-Selection-Verfahren. Ausgehend von einer beliebigen Feature-Basis wird in
jedem Schritt dasjenige Feature ergänzt, das den R²-Wert am stärksten erhöht. Die Schleife endet, sobald kein weiteres
Feature eine Verbesserung bringt. Die Funktion gibt die finale Feature-Liste
einschließlich des Targets zurück.

In [32]:
def get_features(data, base_features, alpha=0.5):

    selected = base_features

    X_total = data.copy()
    y = X_total.pop('sale_price')
    all_features = list(X_total.columns)

    base_score = 0

    while True:

        best_score = 0
        best_feature = None

        # Teste alle noch nicht gewählten Features
        for col in all_features:
            if col not in selected:

                features = selected + [col]
                X = X_total[features]

                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.3, random_state=1
                )

                preprocessor = ColumnTransformer([
                    ('num', StandardScaler(), Selector(dtype_include='number')),
                    ('cat', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False),
                     Selector(dtype_include='object'))
                ])

                model = Ridge(alpha=alpha)
                reg = TransformedTargetRegressor(
                    regressor=model,
                    func=np.log1p,
                    inverse_func=np.expm1
                )

                pipe = make_pipeline(preprocessor, reg)
                pipe.fit(X_train, y_train)
                preds = pipe.predict(X_test)
                score = r2_score(y_test, preds)

                if score > best_score:
                    best_score = score
                    best_feature = col

        if best_score > base_score:
            selected.append(best_feature)
            base_score = best_score
        else:
            break

    return (selected + ['sale_price'])


Als nächstes wenden wir die Funktionen an, um die optimalen Features zu bestimmen, das beste alpha für die ermittelten Features, sowie die Modellgüte.  
Ergebnis wird ein lokales optimum sein, also nicht unbedingt die beste Kombination aus Hyperparameter und Features.

In [33]:
base_features = [b for b in base if b not in ['sale_price']]
features = get_features(df_fe, base_features = base_features)
features

['kms_run',
 'transmission',
 'variant',
 'registered_state',
 'make',
 'model',
 'total_owners',
 'car_rating',
 'age',
 'body_type',
 'avg_rating_by_model',
 'rto_state',
 'fuel_type',
 'age_vs_model_avg',
 'kms_per_age',
 'avg_age_by_model',
 'kms_per_owner',
 'sale_price']

In [34]:
get_alpha_ridge(df_fe[features], low = -1, high = 1, n = 50)

{'transformedtargetregressor__regressor__alpha': np.float64(0.7196856730011519)}

In [35]:
result_5 = get_ridge(df_fe[features], alpha =0.72, return_mae = True)
result_5

(np.float64(0.9043663582952725), np.float64(45783.12850969301))

Regularisierung führt in diesem Fall zu einer deutlich besseren Modellperformance.
Neben Ridge (L2-Regularisierung) könnte auch eine Lasso-Regression (L1-Regularisierung) eingesetzt werden, die Koefﬁzienten mit sehr geringem Einfluss vollständig auf 0 setzen kann und so eine automatische Feature-Selektion ermöglicht.

Für dieses Projekt habe ich mich jedoch bewusst für Ridge entschieden, da es bei stark korrelierten Features stabilere Ergebnisse liefert und in der Praxis häufig die bessere Performance sowie kürzere Berechnungszeiten bietet.

## Übersicht & Interpretation
#### Übersicht Modellperformance
Hier trage ich die Zwischenergebnisse in eine Tabelle ein, um zu vergleichen, welche Performance die einzelnen Schritte gebracht haben.

In [36]:
steps = [
    ('Regression ohne FE', result_1),
    ('Zusätzliche Features', result_2),
    ('Binning', result_3),
    ('Log-Transformation des Targets', result_4),
    ('Ridge Regression', result_5)
]

rows = []
for name, (r2, mae) in steps:
    rows.append((name, round(r2, 4), round(mae, 0)))

df_results = pd.DataFrame(rows, columns=['Modellschritt', 'R² (CV)', 'MAE (CV)'])
df_results.index = df_results.index + 1
df_results


,Modellschritt,R² (CV),MAE (CV)
1,Regression ohne FE,0.8775,49340.0
2,Zusätzliche Features,0.8788,49051.0
3,Binning,0.8818,48853.0
4,Log-Transformation des Targets,0.8952,47026.0
5,Ridge Regression,0.9044,45783.0


Das bedeutet, dass unser Ridge Modell 90,44% der Varianz des Verkauspreises erklärt. Im Schnitt weichen die Schätzungen um 45783 Einheiten (vermutlich indische Rupien) vom tatsächlichen Preis des Autos ab.

#### Interpretation kategorischer Koeffizienten
Zuerst extrahiere ich die Koeffizienten der Ridge-Regression. Dafür nutze ich die zuvor definierte Funktion **get_ridge**. Anschließend sortiere ich die erzeuge Series in kleinere Series, je nach Feature, dem sie zugehörig sind.

In [37]:
important_features = get_ridge(df_fe[features], alpha = 0.72, return_coef = True)

In [38]:
#Aufteilen der Koeffizienten je nach Feature
Variant = important_features[important_features.index.str.startswith('cat__variant')]
Model = important_features[important_features.index.str.startswith('cat__model')]
Make = important_features[important_features.index.str.startswith('cat__make')]
Transmission = important_features[important_features.index.str.startswith('cat__transmission')]
Registered_state = important_features[important_features.index.str.startswith('cat__registered_state')]
Car_rating = important_features[important_features.index.str.startswith('cat__car_rating')]
Body_type = important_features[important_features.index.str.startswith('cat__body_type')]
Fuel_type = important_features[important_features.index.str.startswith('cat__fuel_type')]
Rto_state = important_features[important_features.index.str.startswith('cat__rto_state')]
Nums = important_features[important_features.index.str.startswith('num__')]

betas_cat = [
    ("Variant", Variant),
    ("Model", Model),
    ("Make", Make),
    ("Transmission", Transmission),
    ("Registered_state", Registered_state),
    ("Car_rating", Car_rating),
    ("Body_type", Body_type),
    ("Fuel_type", Fuel_type)
]

Die Koeffizienten, mit denen die Kategorien in unser Modell einfließen, sind jeweils als Faktor zu interpretieren, der auf die Referenzkategorie des selben Features angewendet wird. Diese Referenzkategorie taucht nicht unter den Koeffizienten auf, da sie durch drop = 'frist' beim One-Hot-Encoding entfernt wurden. Verwenden wir also erneut **get_ridge** um diese Referenzkategorien anzuzeigen.

In [39]:
get_ridge(df_fe[features], alpha = 0.72, return_reference = True)

Referenzkategorien:


,Feature,Referenzkategorie
1,transmission,automatic
2,variant,1.0 climber opt amt
3,registered_state,Other
4,make,Other
5,model,3 series
6,car_rating,fair
7,body_type,hatchback
8,rto_state,ap
9,fuel_type,diesel


Als Nächstes stelle ich pro Feature jeweils den höchsten und niedrigsten Koeffizienten, mit denen Kategorien ins Modell einfließen kompakt in einem Dataframe dar. Da das Target logarithmiert wurde, wenden wir exp()-1 an und multiplizieren das mit 100 um einen prozentualen Aufschlag auf die Referenzkategorie als interpretierbares Ergebnis zu bekommen.

In [40]:
def clean(cat_name):
    return cat_name.split('__', 1)[1].split('_', 1)[1].replace('_', ' ')

table = PrettyTable()
table.field_names = [
    'Feature',
    'Highest_Category', 'Coef_Highest',
    'Lowest_Category', 'Coef_Lowest'
]

for name, series in betas_cat:

    highest_cat_raw = series.idxmax()
    lowest_cat_raw = series.idxmin()

    table.add_row([
        name,
        clean(highest_cat_raw), f'{round(np.expm1(series.max())*100, 2)} %',
        clean(lowest_cat_raw),  f'{round(np.expm1(series.min())*100, 2)} %'
    ])

table

Feature,Highest_Category,Coef_Highest,Lowest_Category,Coef_Lowest
Variant,3.0 v 6 premium luxury,125.83 %,c 220 cdi elegance mt,-48.13 %
Model,zen,82.48 %,nano,-50.4 %
Make,mercedes benz,71.21 %,datsun,-33.32 %
Transmission,missing,-8.73 %,manual,-16.08 %
Registered_state,state telangana,31.57 %,state maharashtra,-7.36 %
Car_rating,rating good,7.9 %,rating great,1.34 %
Body_type,type luxury sedan,102.82 %,type sedan,8.05 %
Fuel_type,type electric,-8.39 %,type petrol & cng,-14.09 %


- Im Vergleich zur Referenzvariante **1.0 climber opt amt** ist **3.0 v 6 premium luxury** im Schnitt um 126% teurer
- **c 220 cdi elegance mt** hingegen ist um 48% billiger
- verglichen mit dem Modell **3 series** ist **zen um 82% teurer während **nano** um 50% günstiger ist
- **mercedes benz** ist die teuerste Marke und erwirkt einen Aufschlag von 72% auf die Referenz
- Schaltfahrzeuge **manual** sind um 16% billiger als die Referenz **automatic**
- Überraschenderweiße sind elektrische Fahrzeuge **electric** um 8% billiger als Dieselfahrzeuge, dies liegt wohl an der geringen Datenmenge für electric

#### Interpretation numerischer Koeffizienten
Werfen wir nun einen Blick auf unsere numerischen Features. Zusätzlich zu exp()-1 muss hier beachtet werden, dass die Werte vor dem training unseres Modells standardisiert wurden. Die Koeffizienten sind also als Änderung pro Standardabweichung zu interpretieren. Druch Anwendung von expm1() und Teilen durch die Standardabweichung erhalten wir eine prozentuale Änderung pro erhöhung einer Einhait eines gegebenen numerischen Features.

In [41]:
df_nums = (
    Nums.rename_axis('feature')
        .reset_index()
)

df_nums['feature'] = df_nums['feature'].str.replace(r'^num__', '', regex = True)
df_nums.columns = ['feature', 'coef']
df_nums.index = df_nums.index + 1
df_nums['coef'] = round(df_nums['coef'], 4)

nums = df_nums['feature'].tolist()


std_series = df_fe[nums].std()
df_nums['std'] = std_series.values


pct = np.expm1(df_nums['coef'] / df_nums['std']) * 100

df_nums['perc_change_per_unit'] = pct.apply(lambda x: f"{x:.6f} %")


df_nums = df_nums.drop(columns='std')

df_nums


,feature,coef,perc_change_per_unit
1,age,-0.3430,-10.542055 %
2,age_vs_model_avg,0.0564,20.924984 %
3,kms_run,-0.0372,-0.000085 %
4,avg_rating_by_model,0.0334,14.659222 %
5,total_owners,-0.0321,-5.395883 %
6,avg_age_by_model,-0.0099,-0.424856 %
7,kms_per_owner,-0.0086,-0.000022 %
8,kms_per_age,0.0074,0.000113 %


- pro zusätzlichem Jahr sinkt also der Preis um etwa 11%
- 1000 zusätzliche gefahrene Kilometer verringern den Preis um -0.085%
- Jeder weitere Besitzer verringert den Preis um etwa 5.4%

#### Fazit

In diesem Notebook habe ich ein lineares Basismodell entwickelt und schrittweise durch sinnvolle Feature-Engineering-Methoden und Regularisierung verbessert. Die log-transformierte Ridge-Regression erzielte dabei die stabilsten und genauesten Ergebnisse und dient als interpretierbares Referenzmodell für die weitere Modellierung.

Es wurde aber deutlich, dass das lineare Modell bestimmte nichtlineare Muster nur eingeschränkt erfassen kann und nur bedingt für hochkardinale Daten geeignet sind.
Im nächsten Notebook werde ich daher Gradient-Boosting-Modelle einsetzen, um komplexere Interaktionen und Nichtlinearitäten abzubilden und die Vorhersagegenauigkeit weiter zu steigern.